In [ ]:
imp = open('/home/lindb/pythonimports.py').read()
exec(imp)
from __future__ import division
%matplotlib inline 

In [ ]:
DIR = '/home/lindb/teakettle/data/moran/dependencies'
if not op.exists(DIR):
    os.makedirs(DIR)

In [ ]:
DIR = '/home/lindb/teakettle/data/moran/amat_files'
if not op.exists(DIR):
    os.makedirs(DIR)

def writer(ij):
    i,j = ij
    import os
    from os import path as op
    text = '''
source('/home/lindb/imports.R')

#load dependencies
dname = '/home/lindb/teakettle/data/moran/dependencies/'
files = c()
for (f in list.files(dname)){
    if (endsWith(f,'RDS'))
    {
        sprintf ("loading %%s file",f)
        fname = paste(dname,f,sep="")
        obj = strsplit(f,split='.RDS')[[1]]
        assign(obj,readRDS(fname))    
    }
}

amat <- numeric(0)
for (k in 1:noff)
{
    a <- matrix(0,(nadult+1),(nadult+1)) 
    amat <- append(amat,list(a))
}
    
i <- %s
j <- %s
print ("i=")
print(i)
print("j=")
print(j)


if(i>0) #we want it to run no matter what
{
    print ("starting loop")
    if(i==j & i<(nadult+1)) #this is the diagonal, can't have same sampled mother and father
    {
        unimportant = 1+1 # amat is already 0 on the diagonal, save some time by not looping
#         for (k in 1:noff)
#         {
#             amat[[k]][i,j] <- 0   
#         }
    }
    if(i==j & i==(nadult+1)) #can have two out-of-plot parents
    { 
        nmat <- numeric(0)
        pg1 <- pgout 
        pg1.b <- pgout 
        pg2 <- pgout 
        pg2.b <- pgout
        print ("starting loci loop")
        for(la in 1:nloci) 
        {#Calculate probability of offspring genotype given parental genotypes using global allele frequencies
            #at the la'th locus
            print (la)
            nm <- nmat.func(pg1,pg2,pg1.b,pg2.b,la) #returns a matrix with ncol = nrow = nallele @ la'th locus
            nmat <- append(nmat,list(nm))
        }
        print ("starting offspring loop")
        for(k in 1:noff)
        {
            print (k)
            pq <- 1 #incorrectly overwriting pq is bad, mkay
            og <- rbind(goff[[1]][k,],goff[[2]][k,]) #get the offspring's observed first and second alleles
            og.b <- rbind(goff.r[[1]][k,],goff.r[[2]][k,]) #get any offspring's observed regenotyping results
            print ("starting loci loop")
            for(la in 1:nloci)
            {
                print (la)
                #om = probability of observing the given offspring genotype given all possible true genotypes
                om <- omat.func(og,og.b,la) 
            #pq = probability of observing the given offspring genotype given the observed parental genotypes
                pq <- pq*sum(nmat[[la]]*om)
                #if the posited parents are not the parents, no use in doing futher calculations @ other loci
                if(pq==0) 
                {
                    break 
                }
            }
            print ("writing amat")
            amat[[k]][i,j] <- pq #pq same as above, for the k'th offspring for the i'th and j'th parent
            amat[[k]][j,i] <- pq 
        }# end offspring loop
    }# end two-out-of-plot loop
    if(i!=j) #if not on the diagonal
    { #all other combos... 
        nmat <- numeric(0)
        if(i<(nadult+1)) #if posited parent one of those who were sampled
        {
            #gadult# == nrow=nadult ncol=nloci
            pg1 <- rbind(gadult[[1]][i,],gadult[[2]][i,])  #get first and second alleles for all loci
            pg1.b <- rbind(gadult.r[[1]][i,],gadult.r[[2]][i,]) #get first and second alleles for regenotyping
        } 
        if(j<(nadult+1)) #if posited parent one of those who were sampled
        {
            pg2 <- rbind(gadultF[[1]][j,],gadultF[[2]][j,])
            pg2.b <- rbind(gadultF.r[[1]][j,],gadultF.r[[2]][j,]) 
        }
        if(i==(nadult+1)) #if posited parent is not one of those who were sampled
        { 
            pg1 <- pgout 
            pg1.b <- pgout
        } 
        if(j==(nadult+1)) #if posited parent is not one of those who were sampled
        {
            pg2 <- pgout #put in all zeros
            pg2.b <- pgout #put in all zeros
        }
        print ("starting loci loop")
        for(la in 1:nloci)
        {#Calculate probability of offspring genotype given parental genotypes using global allele frequencies
            #at the la'th locus
            print (la)
            nm <- nmat.func(pg1,pg2,pg1.b,pg2.b,la) 
            nmat <- append(nmat,list(nm))
        }
        print ("starting offspring loop")
        for(k in 1:noff)
        { #pre-calc nmat at all loci for parent pair
            print(k)
            pq <- 1 #incorrectly overwriting is still bad, mkay
            og <- rbind(goff[[1]][k,],goff[[2]][k,]) #get the k'th offspring's 1st and 2nd alleles @ all loci
            og.b <- rbind(goff.r[[1]][k,],goff.r[[2]][k,])
            print ("starting loci loop")
            for(la in 1:nloci)
            {
                print (la)
                #om = probability of observing the given offspring genotype given all possible true genotypes
                om <- omat.func(og,og.b,la)
                #pq = probability of observing the given offspring genotype given the observed parental genotypes
                pq <- pq*sum(nmat[[la]]*om)
                if(pq==0) 
                {
                #if the posited parents are not the parents, no use in doing futher calculations @ other loci
                    break 
                }
            }
            amat[[k]][i,j] <- pq 
            amat[[k]][j,i] <- pq 
        }# end offspring loop
    }#end "all other combos" loop
    
#     track[i,j] <- 1
#     track[j,i] <- 1

} 

print ("writing RDS files")
DIR = '/home/lindb/teakettle/data/amat_outfiles/'
amatfname = paste(DIR,'amat_%s_%s.RDS',sep="")
nmatfname = paste(DIR,'nmat_%s_%s.RDS',sep="")
sprintf("writing %%s",amatfname)
saveRDS(amat,amatfname)
sprintf("writing %%s",nmatfname)
saveRDS(nmat,nmatfname)

''' % (i,
       j,
       str(i).zfill(3),
       str(j).zfill(3),
       str(i).zfill(3),
       str(j).zfill(3)
      )
    DIR = '/home/lindb/teakettle/data/moran/amat_files2'
    if not op.exists(DIR):
        os.makedirs(DIR)
    filE = op.join(DIR,"amat_%s_%s.R" % (str(i).zfill(3),str(j).zfill(3)))
    with open(filE,'wb') as o:
        o.write("%s" % text)
    return(filE)
#dview['writer'] = writer

In [ ]:
from ipyparallel import Client

In [ ]:
rc = Client(profile="new")
dview = rc[:]
lview = rc.load_balanced_view()
len(lview)

In [ ]:
#558*558 = 311,364 comparisons, sigh
jobs = []
for k in np.arange(558): #558 = nadult
    i = k+1 #R doesn't have zeroth elements (lame)
    for w in np.arange(558):
        j = w+1
        jobs.append(lview.apply_async(writer,(i,j)))

In [ ]:
count = 0
for k in jobs:
    if k.r:
        break
count

In [ ]:
dont forget about amat[[k]] <- amat[[k]]/sum(amat[[k]])

In [ ]:
DIR = '/home/lindb/teakettle/data/amat_outfiles/'
if not op.exists(DIR):
    os.makedirs(DIR)

# make sh files to run R files

In [ ]:
DIR = '/home/lindb/teakettle/data/moran/amat_files2/'
rfiles = fs(DIR)
luni(rfiles) #nadult**2

In [ ]:
rfiles[0]

In [ ]:
#realized I only need to run files where i < j 
    #(bc only doing lowertri, reflecting across diagonal in Rscript, diag == 0)
todo = []
for q in range(558):
    i = q+1
    for w in range(558):
        j = w+1
        if i < j:
            filE = '/home/lindb/teakettle/data/moran/amat_files2/amat_%s_%s.R' % (str(i).zfill(3),str(j).zfill(3))
            if op.exists(filE):
                todo.append(filE)
            else:
                print filE

In [ ]:
len(todo)

In [ ]:
#check len(todo)
count = 0
for i in range(558):
    for j in range(i):
        count += 1
count

In [ ]:
# how many r files per sh file?
math.ceil(len(todo)/143)

In [ ]:
DIR = '/home/lindb/teakettle/data/moran/amat_runfiles/'
if not op.exists(DIR):
    os.makedirs(DIR)
cd('/home/lindb/teakettle/data/moran/amat_runfiles/')
fcount = 0
shcount = 0
rcount = 0
for f in sorted(todo):
    if rcount == 0:
        text = '''#!/bin/bash
#$ -N moran%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s

''' % (str(shcount).zfill(3),
       op.dirname(f),
       f)
    else:
        newtext = '''
cd %s
R --no-save < %s
''' % (op.dirname(f),
       f)
        text = text + newtext
    fcount += 1
    rcount += 1
    
    if rcount == 1087 or fcount == len(todo):
        filE = op.join(DIR,'amat_%s.sh' % str(shcount).zfill(3))
        with open(filE,'wb') as o:
            o.write("%s" % text)
        
        rcount = 0
        shcount += 1

In [ ]:
DIR = '/home/lindb/teakettle/data/moran/amat_runfiles/'
cd(DIR) #so .o files don't end up some place weird
for f in fs(DIR):
    !qsub $f